In [8]:
import azureml.core
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
import os
import shutil
import joblib
from azureml.train.automl import AutoMLConfig

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="AutoML-SDK")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-137204
Azure region: southcentralus
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-137204


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Compute target exists')
except ComputeTargetException:
    print('Creating a new compute target')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=5)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=10)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute target
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-04T10:05:58.572000+00:00', 'errors': None, 'creationTime': '2021-02-04T10:05:54.983712+00:00', 'modifiedTime': '2021-02-04T10:06:10.826942+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
ws.datasets.keys()

KeysView({'Houses': DatasetRegistration(id='1d613470-1f54-4c92-bc63-b2042799091a', name='Houses', version=1, description='', tags={})})

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Houses"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # azureml-core of version 1.0.72 or higher is required
        # azureml-dataprep[pandas] of version 1.1.34 or higher is required
        from azureml.core import Workspace, Dataset

        subscription_id = 'a0a76bad-11a1-4a2d-9887-97a29122c8ed'
        resource_group = 'aml-quickstarts-137204'
        workspace_name = 'quick-starts-ws-137204'

        workspace = Workspace(subscription_id, resource_group, workspace_name)

        dataset = Dataset.get_by_name(workspace, name='Houses')
        dataset.to_pandas_dataframe()

df = dataset.to_pandas_dataframe()
df.describe()

,Transactieprijs_m2,BU_g_woz_v2018,BU_g_woz_v2016,WK_g_woz_v2019,BU_g_ink_po_v2017,Woonoppervlakte,BAGbouwjaar,maanden_sinds_jan2004,BUURT_m2_alle_objecten,Woningtype_appartement,...,Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345
count,51314.000000,51314.000000,51314.000000,51314.000000,51314.000000,51314.000000,51314.000000,51314.000000,5.131400e+04,51314.000000,...,50458.0,50774.0,50822.0,51314.000000,51314.000000,48512.0,51263.0,51202.0,51070.0,50961.0
mean,3545.260494,272.382644,220.076568,303.708949,35.010845,101.161009,1952.495245,109.128074,2.822468e+05,0.458627,...,-1.0,-1.0,-1.0,265.703654,277.547700,-1.0,-1.0,-1.0,-1.0,-1.0
std,1237.777275,94.270025,78.045218,66.238606,8.615431,46.006606,64.167687,60.577667,1.913169e+05,0.498290,...,0.0,0.0,0.0,1006.343176,951.008235,0.0,0.0,0.0,0.0,0.0
min,207.428571,-1.000000,-1.000000,177.000000,-1.000000,11.000000,1250.000000,0.000000,2.149800e+04,0.000000,...,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
25%,2625.000000,212.000000,164.000000,250.000000,29.100000,74.000000,1926.000000,52.000000,1.827970e+05,0.000000,...,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
50%,3294.117647,265.000000,214.000000,328.000000,33.900000,93.000000,1961.000000,119.000000,2.450660e+05,0.000000,...,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
75%,4200.000000,326.000000,262.000000,354.000000,41.100000,120.000000,1997.000000,162.000000,3.260010e+05,1.000000,...,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
max,23890.909091,771.000000,647.000000,403.000000,63.800000,1145.000000,2020.000000,203.000000,1.649683e+06,1.000000,...,-1.0,-1.0,-1.0,5326.219181,4895.161290,-1.0,-1.0,-1.0,-1.0,-1.0


In [6]:
dataset.take(5).to_pandas_dataframe()

,Transactieprijs_m2,BU_g_woz_v2018,BU_g_woz_v2016,WK_g_woz_v2019,BU_g_ink_po_v2017,Woonoppervlakte,BAGbouwjaar,maanden_sinds_jan2004,BUURT_m2_alle_objecten,Woningtype_appartement,...,Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345
0,2700.000000,156,137,219,28.1,40,1962,1,290265,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1
1,1725.000000,149,125,219,23.1,80,1956,1,413897,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1
2,2350.000000,189,151,247,28.1,60,1979,1,184252,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1
3,2875.000000,213,167,250,26.9,48,1919,1,116432,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1
4,2057.142857,156,137,219,28.1,70,1962,1,290265,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1


In [10]:
project_folder = './AutoML_SDK'

automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric": 'spearman_correlation'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="Transactieprijs_m2",   
                             path = project_folder,
                             validation_size = 0.2,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [11]:
run = exp.submit(config=automl_config, show_output=True)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_7f9ccfba-8cb6-4e81-b64a-9f25fc3e48ea

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:  

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary de

{'runId': 'AutoML_7f9ccfba-8cb6-4e81-b64a-9f25fc3e48ea',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-04T10:14:10.651348Z',
 'endTimeUtc': '2021-02-04T10:46:07.163031Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'spearman_correlation',
  'train_split': '0.2',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-SDK","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-137204","workspace_name":"quick-starts-ws-137204","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"spearman_correlation","task_type":"regression","data_script":null,"validation_size":0.2,"n_cross_validations":null,"y_min":null,"y_max":nul

In [12]:
# Retrieve and save your best automl model.
best_run, best_model = run.get_output()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [13]:
print(best_run)

Run(Experiment: AutoML-SDK,
Id: AutoML_7f9ccfba-8cb6-4e81-b64a-9f25fc3e48ea_38,
Type: azureml.scriptrun,
Status: Completed)


In [14]:
print(best_model)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [15]:
best_run.register_model(model_name = "automl_udacity_project.pkl", model_path = './outputs/')
print(best_model._final_estimator)

PreFittedSoftVotingRegressor(estimators=[('0',
                                          Pipeline(memory=None,
                                                   steps=[('maxabsscaler',
                                                           MaxAbsScaler(copy=True)),
                                                          ('lightgbmregressor',
                                                           LightGBMRegressor(boosting_type='gbdt',
                                                                             class_weight=None,
                                                                             colsample_bytree=1.0,
                                                                             importance_type='split',
                                                                             learning_rate=0.1,
                                                                             max_depth=-1,
                                                                  

In [25]:
compute_target.delete()